## imports

In [1]:
import os,time,sys
import shutil
import argparse


import ghtest.param_suggestor as ps
from ghtest import make_test, scan, suggest, write_module
from ghtest.tests_writer import TestArtifact
from ghtest import cassette_sanitizer as sanitizer

## format

In [2]:
# set print behavior
from IPython.display import display, Javascript
display(Javascript("""
document.querySelectorAll('.jp-OutputArea-child').forEach(e => e.style.maxHeight = 'none');
"""))

<IPython.core.display.Javascript object>

## constants

In [3]:
# input
cassette_dir='tests/cassettes'
test_dir='testdata_tests/'
#src_folder='../temp_folder/src/apigh/'
src_folder='testdata'
#test_folder='../temp_folder/tests/'
safe_switch='GHTEST_ASSUME_SAFE'
is_unsafe='1'
remove_current_tests=True
remove_all_tests=True
remove_cassettes=True
remove_history=True
literal_only=True

## helpers

In [4]:
def _parse_kv(s):
    if '=' not in s:
        return
    k, v = s.split('=', 1)
    if not k:
        return
    return k, v

## switches

In [5]:
#os.environ[safe_switch]=is_unsafe

## remove stale test data

In [6]:
# remove param_db and param history
def _remove_history(vb=0):
    remove_param_db=True
    param_db_path='../../.local/state/ghtest/param_db.json'
    param_hist_path='../../.local/state/ghtest/param_history.json'
    pps=[param_db_path, param_hist_path]
    for pt in pps:
        if os.path.isfile(pt):
            os.remove(pt)
            bn=os.path.basename(pt)
            if vb:print(f'{bn} removed')

## scan

In [7]:
# scan 
#scs=scan(src_folder)
#sct=scan(test_folder)len(scs)#, len(sct)

### filter non-api and test methods

In [8]:
def _filter_functions(scs, filter_private=False, filter_tests=False, vb=0):
    funcs=scs
    if filter_private:
        funcs=[f for f in funcs if not f.qualname.startswith('_')]
    if filter_tests:
        funcs=[f for f in funcs if not f.qualname.startswith('test_')]
    if vb:
        for func in funcs:
            print(func.qualname )
    return funcs

## suggest 

In [9]:
def _suggest_params(scs, tcs=None, literal_only=True, vb=0):
    sps=[]
    lps=0
    for func in scs:
        sp=suggest(func,test_functions=tcs, literal_only=literal_only)
        if vb>1:
            print(sp.module, sp.qualname)
        for ps in sp.param_sets:
            pass
            if vb>1:
                print(ps)
            lps+=len(ps)
        sps.append(sp)
        if vb>1:
            print()
    if vb:
        print(f'{lps} parameter sets')
    return sps

### change parameters

In [10]:
def _change_params(sps, param_update=None, vb=0):
    if not param_update:
        param_update={}
    nup=0
    for sp in sps:
        updated_param_set=[]
        for op in sp.param_sets:
            for k,v in param_update.items():
                if k in op:
                    op=op|param_update
                    nup+=1
            updated_param_set.append(op)
        sp.param_sets=updated_param_set
        if vb>1:
            print(sp.qualname)
            print(sp.param_sets)
            print()
    if vb:
        print(f'{nup} param sets updated')

## make tests

### remove old cassettes and tests

In [11]:
def _make_tests_and_remove_old_cassettes(sps, cassette_dir, vb=0):
    gts=[]
    cfs=[os.path.join(cassette_dir,f) for f in os.listdir(cassette_dir)]
    nr=0
    for sp in sps:
        gt=make_test(suggestion=sp, cassette_dir=cassette_dir)
        gts.append(gt)
        cp=gt.cassette_path
        bn=os.path.splitext(cp)[0]
        for f in cfs:
            if remove_cassettes and bn in f and os.path.isfile(f):
                os.remove(f)
                nr+=1
    if vb:
        print(f'{nr} files deleted from {cassette_dir}')
    return gts

In [12]:
# remove previous tests
def _remove_old_tests(funcs, test_dir, vb=0):
    funcnames=[f.qualname  for f in funcs]
    nf,nd=0,0
    for file in os.listdir(test_dir):
        path=os.path.join(test_dir, file)
        if os.path.isfile(path):
            if remove_all_tests or any(f in file for f in funcnames) and remove_current_tests:
                os.remove(path)
                nf+=1
        elif os.path.isdir(path):
            shutil.rmtree(path)
            nd+=1
    if vb:
        print(f'{nf} files, {nd} folders deleted from {test_dir}')

### create cassettes

In [13]:
def _run_tests(gts, interactive=True, vb=0):
    trs=[]
    for gt in gts:
        try:
            tr=gt.test_callable(interactive=interactive)
            trs.append(tr)
        except Exception as e:
            if vb:
                print(str(e))
    return trs

### sanitize cassettes

In [14]:
def _sanitize_cassettes(cassette_dir, vb=0):
    args=[cassette_dir,]
    r=sanitizer.main(args, vb=vb)

### record results

In [15]:
if False:
    trs=[]
    for gt in gts:
        try:
            tr=gt.test_callable()
            trs.append(tr)
        except Exception as e:
            print(str(e))

### write tests

In [16]:
def _write_tests(sps, trs, simple_checks=False, vb=0):
    artifacts=[]
    rrs=[]
    for sp,tr in zip(sps, trs):
        artifacts.append(TestArtifact(suggestion=sp, run=tr))
    r=write_module(artifacts=artifacts, output_dir=test_dir, include_return_summary=not simple_checks)
    
    ltm=len(r.test_modules)if 'test_modules' in r.__dict__ else 0
    lsm=len(r.scenario_modules)if 'scenario_modules' in r.__dict__ else 0
    if vb:
        print(f'saved {ltm} test module(s), {lsm} scenario module(s)')
    return r

In [17]:
def test_args(argv=None, print_args=False):
    parser = argparse.ArgumentParser(description="create tests")
    parser.add_argument("--cassette_dir", help="folder for cassette files", default='tests/cassettes')
    parser.add_argument("--test_dir", help="folder to store generated tests")
    parser.add_argument("--src_folder", help="src folder under test")
    parser.add_argument("--example_test_folder", help="example tests for src under test")
    parser.add_argument("--use_unsafe", action="store_true", help="set to execute poetentially desctructive functions without confirmation")

    parser.add_argument("--simple_checks", action="store_true", help="ignore attributes of return values")
    parser.add_argument("--keep_current_tests", action="store_false", help="do not remove existing tests for currently tested functions")
    parser.add_argument("--remove_all_tests", action="store_true", help="remove all existing tests")
    parser.add_argument("--remove_cassettes", action="store_true", help="rewrite cassettes for functions under test")
    parser.add_argument("--remove_history", action="store_true", help="remove existing user-wide param db")
    parser.add_argument("--literal_only", action="store_true", help="restrict tests to literal values found by scanner")
    
    parser.add_argument('-v', action='count', default=0, help='set verbosity level')

    parser.add_argument("--interactive", action="store_true", help="explicitly confirm desctructive action if user_unsafe is not set")
    parser.add_argument("--dry_run", action="store_true", help="report changes without executing functions or writing files.")
    parser.add_argument("--keep_test_functions", action="store_true", help="include functions starting with test_ in functions under test")
    parser.add_argument("--keep_private_functions", action="store_true", help="include functions starting with _ in functions under test")
    parser.add_argument("--param_update", action="append", help="change specified parameters, use optionally repeated,form: key=value")    
    
    args = parser.parse_args(argv)
    if print_args:
        return args
    vb=args.v
    vb,vb1,vb2 = max(0,vb), max(0,vb-1), max(0,vb-2)
    if vb1:print(f'verbosity: {vb}')
    
    dry_run=args.dry_run
    interactive=args.interactive
    
    simple_checks=args.simple_checks
    src_folder=args.src_folder
    example_test_folder=args.example_test_folder
    
    if dry_run:
        if vb1:print('dry run')
    if not dry_run:
        os.environ[safe_switch]='1' if args.use_unsafe else '0'
    if args.remove_history and not dry_run:
        if vb1:
            print('remove_history(vb=vb)')
        _remove_history(vb=vb2)
    else:
        if vb1:
            print(f'no history db removed')

    # scan
    if not src_folder:
        print('fatal: src_folder missing')
        return
    else:
        scs=scan(src_folder)
    
    tcs=None
    if example_test_folder:
        tcs=scan(example_test_folder) 
    if vb:
        if scs:
            if vb1:
                print(f'scanned {len(scs)} functions from {src_folder}')
        if tcs:
            if vb1:
                print(f'scanned {len(tcs)} functions from {example_test_folder}')

    # filter functions
    if vb1:
        print()
        print('functions:')
    filter_private=not args.keep_private_functions
    filter_tests=not args.keep_test_functions
    scs=_filter_functions(scs, filter_private=filter_private, filter_tests=filter_tests, vb=vb2)
    if scs and vb1:
        print()
        print(f'{len(scs)} source functions remaining after filtering {"" if args.keep_private_functions else "private functions"}{"" if args.keep_test_functions else ", test functions"}')
    if tcs:
        if vb1:
            print()
        tcs=_filter_functions(tcs, filter_private=args.keep_private_functions, vb=vb)
        if vb1:
            print(f'{len(tcs)} functions remaining after filtering from {example_test_folder}')
    
    # suggest
    if vb1:
        print()
    sps=_suggest_params(scs, tcs=None, literal_only=literal_only, vb=vb2)
    if sps and vb1:
        print(f'params for {len(sps)} functions')

    # change parameters
    param_update=args.param_update
    if param_update:
        if vb1:
            print()
            print(f'param_update: {param_update}')
        param_update=[_parse_kv(k) for k in param_update]
        param_update={k:v for (k,v) in param_update if k}
        if vb1: 
            print(f'param_update: {param_update}')
            print()
        # this changes sps contents, no return value
        _change_params(sps, param_update=param_update, vb=vb2)
    
    if not dry_run:
        # make tests and remove old cassettes
        cassette_dir=args.cassette_dir
        gts=_make_tests_and_remove_old_cassettes(sps=sps, cassette_dir=cassette_dir, vb=vb2)
        
        # remove old tests
        test_dir=args.test_dir
        if not test_dir:
            print('fatal: no test_dir')
            return
        _remove_old_tests(funcs=scs, test_dir=test_dir, vb=vb2)

        if vb1:
            print(f'interactive: {interactive}')
        
        # create new cassettes
        trs=_run_tests(gts, interactive=interactive, vb=vb2, )

        # sanitize
        _sanitize_cassettes(cassette_dir=cassette_dir, vb=vb2)
    
        # record results
        trs=_run_tests(gts, interactive=interactive, vb=vb2)
    
        # write tests
        r=_write_tests(sps=sps, trs=trs, simple_checks=simple_checks, vb=vb2)
    else:
        r=None
    if r is None:
        if vb:
            print(f'dry_run: no tests generated')
    else:
        ltm=len(r.test_modules)if 'test_modules' in r.__dict__ else 0
        lsm=len(r.scenario_modules)if 'scenario_modules' in r.__dict__ else 0
        if vb:
            print(f'saved {ltm} test module(s), {lsm} scenario module(s)')
    return r

In [18]:
# paths
print_args=True

src_folder='testdata'
example_test_folder= '../temp_folder/tests/tests/cassettes/'
test_dir = 'testdata_tests'
cassette_dir='tests/cassettes/'
paths={'src_folder':src_folder, 'example_test_folder':example_test_folder, 'test_dir':test_dir, 'cassette_dir':cassette_dir}
# switches
dry_run=False
interactive=False
use_unsafe=False

keep_current_tests=True
remove_all_tests=False
remove_cassettes=False
remove_history=True
literal_only=True

simple_checks=True
keep_test_functions=False
keep_private_functions=False

switches={'dry_run':dry_run,'interactive':interactive,'use_unsafe':use_unsafe,'keep_current_tests':keep_current_tests,
          'remove_all_tests':remove_all_tests,'remove_cassettes':remove_cassettes,'remove_history':remove_history,
          'literal_only':literal_only,'keep_test_functions':keep_test_functions,
          'keep_private_functions':keep_private_functions, 'simple_checks':simple_checks}

# repeat args
param_update={'name':'coron7'}
repeat_args={'param_update':param_update}

# verbosity
vb=1
verbosity:{'vb':vb}


args=[]

for k,v in paths.items():
    args.append(f'--{k}')
    args.append(f'{v}')
for k,v in switches.items():
    if v:
        args.append(f'--{k}')
for ra,d in repeat_args.items():
    for k,v in d.items():
        args.append(f'--{ra}')
        args.append(f'{k}={v}')
if vb:
    args.append(f'-{vb*"v"}')

if print_args:
    r=test_args(args, print_args=True)
    print('args:')
    for arg in r.__dir__():
        if not arg.startswith('_'):
            print(arg, eval(f'r.{arg}'))
    print()
r=test_args(args)

args:
cassette_dir tests/cassettes/
test_dir testdata_tests
src_folder testdata
example_test_folder ../temp_folder/tests/tests/cassettes/
use_unsafe False
simple_checks True
keep_current_tests False
remove_all_tests False
remove_cassettes False
remove_history True
literal_only True
v 1
interactive False
dry_run False
keep_test_functions False
keep_private_functions False
param_update ['name=coron7']

saved 4 test module(s), 0 scenario module(s)
